In [45]:
import numpy as np
import torch
import os
import h5py
import pickle
import fnmatch
import cv2
import sys
from time import time
from torch.utils.data import TensorDataset, DataLoader
np.set_printoptions(suppress=True)


In [46]:
import kdl_parser_py.urdf


In [47]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 64
policy_class = "EPACT"

Found 51 hdf5 files


In [49]:
train_dataset = EpisodicDataset(dataset_path_list, camera_names, norm_stats, train_episode_ids, train_episode_len, chunk_size, policy_class)


URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  True
augment_images:  False
ee_pose:  (65, 6)
ee_pose:  [[ 0.    -0.     0.     0.     0.    -0.   ]
 [-0.001 -0.001 -0.    -0.     0.    -0.005]
 [ 0.    -0.001  0.    -0.     0.    -0.004]
 [-0.    -0.001  0.    -0.     0.    -0.005]
 [-0.    -0.001  0.    -0.     0.    -0.006]
 [-0.001 -0.002 -0.    -0.     0.    -0.009]
 [-0.001 -0.002 -0.    -0.     0.    -0.009]
 [-0.001 -0.002  0.    -0.     0.    -0.008]
 [-0.001 -0.003  0.    -0.     0.    -0.011]
 [-0.001 -0.003  0.    -0.     0.    -0.012]
 [-0.002 -0.004 -0.    -0.     0.    -0.016]
 [-0.002 -0.005  0.    -0.     0.    -0.018]
 [-0.001 -0.005  0.    -0.     0.    -0.018]
 [-0.002 -0.006  0.    -0.     0.    -0.02 ]
 [-0.002 -0.006 -0.    -0.     0.    -0.023]
 [-0.003 -0.007 -0.    -0.     0.    -0.027]
 [-0.003 -0.008  0.    -0.     0.    -0.03 ]
 [-0.003 -0.009  0.    -0.     0.    -0.032]
 [-0.003 -0.01   0.    -0.     0.    -0.036]
 [-

In [51]:
index = 50
image_data, qpos_data, action_data, is_pad, end_pose_data = train_dataset[0]
print("image_data: ", image_data.shape)
print("qpos_data: ", qpos_data.shape)
print("action_data: ", action_data.shape)
print("end_pose_data: ", end_pose_data.shape)

ee_pose:  (65, 6)
image_data:  torch.Size([3, 3, 480, 640])
qpos_data:  torch.Size([4])
action_data:  torch.Size([64, 5])
end_pose_data:  torch.Size([64, 5])


In [33]:

import transformations as tf

def batch_transform_to_xyzyrp_transformations(batch_transform):
  """
  Converts a batch of transformation matrices (65, 4, 4) to a batch of (x, y, z, roll, pitch, yaw) (65, 6) using transformations library.

  Note: This approach might have limitations and is not recommended as the primary method due to potential limitations in rotation order and axes supported.

  Args:
      batch_transform: A NumPy array of shape (batch_size, 4, 4) representing the batch of transformation matrices.

  Returns:
      A NumPy array of shape (batch_size, 6) containing the translation (x, y, z) and Euler angles (roll, pitch, yaw) for each transformation matrix.
  """
  batch_xyzyrp = np.zeros((batch_transform.shape[0], 6))
  for i in range(batch_transform.shape[0]):
    matrix = batch_transform[i]
    translation = matrix[:-1, 3]  # Extract translation (might be limited axes)
    euler_angles = tf.euler_from_matrix(matrix, axes='sxyz')  # Might be limited axes

    batch_xyzyrp[i] = np.concatenate([translation, euler_angles])
  return batch_xyzyrp

In [34]:
trans_mats = np.asarray([np.diag([1, 1, 1, 1])] * 64)
print(trans_mats.shape)

(64, 4, 4)


In [35]:
batch_euler_angles = batch_transform_to_xyzyrp_transformations(trans_mats)
batch_euler_angles

array([[ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0